In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

In [2]:
posts_df = pd.read_csv('data/posts.csv')
posts_df.head()

,Unnamed: 0,age,comment,likes/views,link,promo,user
0,0,2 DAYS AGO,found my new favorite park!,"405,059 likes",/BytNlrQhRx8/,0,chrissyteigen
1,1,3 DAYS AGO,Happy bebe!,"1,739,218",/Byqz8uZh73s/,0,chrissyteigen
2,2,5 DAYS AGO,coated in a paste of fresh garlic and filled w...,"2,931,603",/BymErW1B9eL/,0,chrissyteigen
3,3,5 DAYS AGO,this kid,"371,095 likes",/Byl-aHjBXFX/,0,chrissyteigen
4,4,JUNE 8,home tomorrow 😩,"859,039 likes",/ByduG0BB_A3/,0,chrissyteigen


In [3]:
posts_df.shape

(1065, 7)

In [4]:
target = posts_df.promo
data = posts_df['comment'].map(word_tokenize).values

In [5]:
total_vocabulary = set(word for comment in data for word in comment)
print("There are {} unique tokens in our dataset.".format(len(total_vocabulary)))

There are 8994 unique tokens in our dataset.


In [6]:
glove = {}
with open('data/glove_data/glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [7]:
glove['cool']

array([-0.65575 ,  0.45659 , -0.16748 , -0.58345 , -0.23073 , -0.78348 ,
       -0.23166 , -0.022452, -0.57968 ,  0.526   , -0.2214  ,  0.17614 ,
        0.46513 ,  0.79142 ,  0.017403,  1.0879  ,  0.24418 ,  0.27523 ,
       -0.26452 , -1.0389  ,  0.014045,  0.68459 ,  0.98151 ,  0.21561 ,
        0.36278 , -0.51819 , -0.40552 ,  1.349   ,  1.5399  ,  0.60541 ,
        2.6604  ,  0.074535, -0.076292,  0.12501 , -0.026268,  0.16843 ,
       -0.41844 ,  0.44505 ,  0.25033 , -1.1557  ,  0.24575 ,  0.41847 ,
       -0.10633 , -0.28433 ,  0.51215 ,  0.51371 ,  0.53004 , -0.889   ,
        0.054744,  0.78793 ], dtype=float32)

#### Create Mean Word Embeddings

In [8]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

#### Using Pipelines

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [10]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [11]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
/Users/sherzyang/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sherzyang/anaconda3/envs/learn-env/lib/python3.6/site-p

In [12]:
scores

[('Random Forest', 0.6281722128962179),
 ('Support Vector Machine', 0.5624321121753728),
 ('Logistic Regression', 0.5877851288634344)]

#### Deep Learning with word embeddings

In [13]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

Using TensorFlow backend.


In [14]:
y = pd.get_dummies(target).values

In [15]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(posts_df.comment))
list_tokenized_headlines = tokenizer.texts_to_sequences(posts_df.comment)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [16]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [45]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(20000, embedding_size)(input_)
x = Bidirectional(LSTM(25, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 2 different possible classes, so we use 2 neurons in our output layer
x = Dense(2, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 100, 50)           30800     
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_9 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 102       
Total para

In [48]:
model.fit(X_t, y, epochs=2, batch_size=32, validation_split=0.1)

Train on 958 samples, validate on 107 samples
Epoch 1/2
958/958 [==============================] - 6s 6ms/step - loss: 0.6850 - acc: 0.5543 - val_loss: 0.6657 - val_acc: 0.6075
Epoch 2/2
958/958 [==============================] - 4s 4ms/step - loss: 0.6604 - acc: 0.6033 - val_loss: 0.6426 - val_acc: 0.6168


In [49]:
model.fit(X_t, y, epochs=10, batch_size=32, validation_split=0.1)

Train on 958 samples, validate on 107 samples
Epoch 1/10
958/958 [==============================] - 4s 4ms/step - loss: 0.5384 - acc: 0.7944 - val_loss: 0.5616 - val_acc: 0.7103
Epoch 2/10
958/958 [==============================] - 4s 4ms/step - loss: 0.2780 - acc: 0.9228 - val_loss: 1.5428 - val_acc: 0.4112
Epoch 3/10
958/958 [==============================] - 4s 4ms/step - loss: 0.1886 - acc: 0.9415 - val_loss: 0.7680 - val_acc: 0.7009
Epoch 4/10
958/958 [==============================] - 3s 4ms/step - loss: 0.0836 - acc: 0.9843 - val_loss: 1.0587 - val_acc: 0.7383
Epoch 5/10
958/958 [==============================] - 4s 4ms/step - loss: 0.0537 - acc: 0.9823 - val_loss: 1.0669 - val_acc: 0.6636
Epoch 6/10
958/958 [==============================] - 3s 4ms/step - loss: 0.0528 - acc: 0.9812 - val_loss: 1.1426 - val_acc: 0.6916
Epoch 7/10
958/958 [==============================] - 4s 4ms/step - loss: 0.0156 - acc: 0.9979 - val_loss: 1.3458 - val_acc: 0.7009
Epoch 8/10
958/958 [==========

In [50]:
import pickle 
test_df = pickle.load(open('data/testing_posts.pkl', 'rb'))

In [51]:
list_tokenized_headlines = tokenizer.texts_to_sequences(test_df.comment)
testing_X = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [52]:
testing_y = pd.get_dummies(test_df['promo']).values

In [53]:
model.evaluate(testing_X, testing_y, verbose=True)

248/248 [==============================] - 0s 839us/step


[1.4726300008835331, 0.6975806470840208]